In [1]:
import pandas as pd
import pybamm
import pybop

In [2]:
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        # "Current function [A]": 5,
        # "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

In [6]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy() * -1, 
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

In [7]:
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[1, 1500],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[0, 25000],
    ),
)

In [8]:
model.build(
    initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]}
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

In [9]:
optim = pybop.PSO(
    cost,
    sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
    max_unchanged_iterations=30,
    max_iterations=100,
)
results = optim.run()

Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [5.29999650e-03 4.93386683e-03 5.63603934e-03 3.16833674e+02
 2.15959939e+03]
  Optimised parameters: [1.82811861e-02 1.65463318e-02 1.08221516e-02 1.35464843e+00
 3.10427956e+03]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.08201698637918063
  Optimisation time: 144.35420870780945 seconds
  Number of iterations: 90
  Number of evaluations: 557
  SciPy result available: No
  PyBaMM Solution available: Yes


In [10]:
pybop.plot.quick(problem, problem_inputs=results.x, title="Optimised Comparison");

In [12]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

In [ ]:
import pandas as pd
import pybamm
import pybop

# Constant temperature for all extractions (modify as needed)
CONSTANT_TEMP = 25  # [°C]

# Set up the base parameter values for the ECM
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update({
    "Cell capacity [A.h]": 5,
    "Nominal cell capacity [A.h]": 5,
    "Element-1 initial overpotential [V]": 0,
    "Upper voltage cut-off [V]": 4.2,
    "Lower voltage cut-off [V]": 3.0,
    "R0 [Ohm]": 1e-3,
    "R1 [Ohm]": 2e-4,
    "C1 [F]": 1e4,
    "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values["Open-circuit voltage [V]"],
})

# Use a single RC element since we are estimating only R0, R1 and C1.
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 1},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

# Define initial guesses and bounds for the three parameters
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter("R0 [Ohm]",
                    prior=pybop.Gaussian(r_guess, r_guess / 10),
                    bounds=[0, 0.2]),
    pybop.Parameter("R1 [Ohm]",
                    prior=pybop.Gaussian(r_guess, r_guess / 10),
                    bounds=[0, 0.2]),
    pybop.Parameter("C1 [F]",
                    prior=pybop.Gaussian(500, 100),
                    bounds=[1, 1500]),
)

# Prepare a list to collect LUT entries for each pulse
lut_entries = []

# Loop over the 10 pulses (pulse_0 to pulse_9)
for pulse in range(10):
    # Build the file path based on pulse index
    file_loc = f"../hppc_lut/G1/battery_G1_cycle_2_pulse_{pulse}_lut.csv"
    
    # Read the extraction data
    df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
    df = df.drop_duplicates(subset=["Time"], keep="first")
    
    # Build the dataset for the estimation. Note: the sign for current is reversed.
    dataset = pybop.Dataset({
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy() * -1,
        "Voltage [V]": df["Voltage"].to_numpy(),
    })
    
    # Use the first voltage value as the initial open-circuit voltage
    initial_voltage = df["Voltage"].to_numpy()[0]
    model.build(initial_state={"Initial open-circuit voltage [V]": initial_voltage})
    
    # Define the fitting problem
    problem = pybop.FittingProblem(model, parameters, dataset)
    
    # Define the cost function (here, the sum of squared errors)
    cost = pybop.SumSquaredError(problem)
    
    # Run the optimization using Particle Swarm Optimization
    optim = pybop.PSO(
        cost,
        sigma0=[1e-3, 1e-3, 50],
        max_unchanged_iterations=30,
        max_iterations=100,
    )
    results = optim.run()
    
    # Extract the estimated parameter values.
    # (Assuming results.best_parameters is a dictionary keyed by parameter names)
    est_R0 = results.x[0]
    est_R1 = results.x[1]
    est_C1 = results.x[2]
    
    # For the LUT, compute representative voltage and current values (e.g., the average)
    avg_voltage = df["Voltage"].mean()
    avg_current = df["Current"].mean()
    
    # Calculate the state-of-charge (SoC) for this pulse.
    # Starting at 1.0 (100%) and decreasing by 0.05 per pulse.
    soc = 1.0 - pulse * 0.05
    
    # Append the results as a new row in the LUT
    lut_entries.append({
        "SoC": soc,
        "R0 [Ohm]": est_R0,
        "R1 [Ohm]": est_R1,
        "C1 [F]": est_C1,
        "Voltage [V]": avg_voltage,
        "Current [A]": avg_current,
        "Temperature [C]": CONSTANT_TEMP,
    })

# Create the lookup table as a pandas DataFrame
lut_df = pd.DataFrame(lut_entries)

# Save the LUT to a CSV file (modify the filename/path as needed)
lut_df.to_csv("battery_ecm_parameter_lut.csv", index=False)
print("Lookup table saved as battery_ecm_parameter_lut.csv")


In [9]:
import pandas as pd
import numpy as np
import pybamm
import pybop

# Load your CSV pulse data
file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv"
df = pd.read_csv(file_loc, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

# PARAMETERS: set cell capacity (convert Ah to A.s)
cell_capacity_Ah = 5
cell_capacity_As = cell_capacity_Ah * 3600  # e.g. 18000 A.s

# --- STEP 1: Compute SoC ---
# We assume an initial SoC of 1.0 (fully charged) and use coulomb counting.
# (Make sure your sign convention is consistent. Here we assume that a negative current in the CSV indicates discharge.)
# Create a time-step array (in seconds)
time = df["Time"].to_numpy()
# Compute time differences (dt)
dt = np.diff(np.concatenate(([time[0]], time)))
# Use the original CSV current (if discharge current is negative, then SoC decreases)
# Integrate current over time to get charge removed (in coulombs)
charge_removed = np.cumsum(df["Current"].to_numpy() * dt)
# Compute SoC: note that charge_removed is negative for discharge so we add it (or subtract its absolute value)
df["SoC"] = 1.0 + (charge_removed / cell_capacity_As)

# --- STEP 2: Define the desired SoC grid ---
desired_soc = np.arange(0, 1.01, 0.05)  # from 0 to 1 in 0.05 increments

# Prepare a list to hold LUT entries
lut_entries = []

# It’s assumed that the temperature and current are constant for the pulse.
# If you have temperature data, you can replace the fixed value below.
temperature = -20

# --- Set up the common model and parameter set (as in your notebook) ---
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": cell_capacity_Ah,
        "Nominal cell capacity [A.h]": cell_capacity_Ah,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values["Open-circuit voltage [V]"],
    }
)
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

# Define the fitting parameters (as in your notebook)
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[1, 1500],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[0, 25000],
    ),
)

# --- STEP 3: Loop over desired SoC values ---
# For each desired SoC value, find the nearest index in the data and take a short window around it
window_size = 20  # adjust as needed (number of points on each side)

for soc_target in desired_soc:
    # Find the index where the measured SoC is closest to the target
    idx = (np.abs(df["SoC"] - soc_target)).idxmin()
    
    # Define a window around this index (taking care of boundaries)
    start_idx = max(0, idx - window_size)
    end_idx = min(len(df), idx + window_size)
    window = df.iloc[start_idx:end_idx].copy()
    
    # Build the dataset for fitting.
    # Note: When building the dataset for pybop, we typically want a current function that is positive for charging.
    # In your notebook you multiplied by -1, so we do the same here.
    dataset = pybop.Dataset({
        "Time [s]": window["Time"].to_numpy(),
        "Current function [A]": window["Current"].to_numpy() * -1,
        "Voltage [V]": window["Voltage"].to_numpy(),
    })
    
    # Build the model with an initial state taken from the start of the window.
    model.build(initial_state={"Initial open-circuit voltage [V]": window["Voltage"].iloc[0]})
    
    # Set up the fitting problem and cost function
    problem = pybop.FittingProblem(model, parameters, dataset)
    cost = pybop.SumSquaredError(problem)
    
    # Run the optimizer (here using PSO as in your example)
    # You might want to reduce iterations for a quicker LUT computation, then refine later.
    optim = pybop.PSO(
        cost,
        sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
        max_unchanged_iterations=30,
        max_iterations=100,
    )
    results = optim.run()
    
    # Extract the optimized parameters.
    # The ordering is given by the order you defined in 'parameters'.
    R0_opt = results.x[0]
    R1_opt = results.x[1]
    # results.x[2] is R2, but we are focusing on C1 here.
    C1_opt = results.x[3]
    
    # For the LUT, record Temperature, (measured) Current, SoC, and C1.
    # Here, we use the current at the center of the window (using the original sign, so e.g. -400 A).
    current_measured = window["Current"].iloc[len(window)//2]
    
    lut_entries.append({
        "Temperature [degC]": temperature,
        "Current [A]": current_measured,
        "SoC": soc_target,
        "C1 [F]": C1_opt,
        # If desired, you could also record R0 and R1:
        # "R0 [Ohm]": R0_opt,
        # "R1 [Ohm]": R1_opt,
    })

# --- STEP 4: Create and display the LUT DataFrame ---
lut_df = pd.DataFrame(lut_entries)
print(lut_df)

# Optionally, save the LUT to a CSV file:
lut_df.to_csv("hppc_lut_results.csv", index=False)

SyntaxError: invalid syntax (3920774325.py, line 1)